In [ ]:
import os
import pandas as pd
from glob import glob

# データを保存するフォルダ
data_dir = 'stocks/stocks/'
# 複数のCSVファイルを読み込んで一つのDataFrameにまとめる
all_files = glob(os.path.join(data_dir, "*.csv"))
df_list = [pd.read_csv(file) for file in all_files]
data = pd.concat(df_list, ignore_index=True)

# 日付データの型変換
data['Date'] = pd.to_datetime(data['Date'])
# 各銘柄に対して一意なIDを割り当てる（銘柄名をIDにするか、もしくはファイル名からIDを生成する）
data['Stock_ID'] = [os.path.basename(file).replace('.csv', '') for file in all_files for _ in range(len(pd.read_csv(file)))]

# TFTに必要な形式に変換（通常はlong format）
data = data[['Stock_ID', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
data = data.rename(columns={'Date': 'time_idx', 'Close': 'target'})

# time_idx（時間のインデックス）を生成する
data['time_idx'] = data.groupby('Stock_ID')['time_idx'].rank(method='dense').astype(int) - 1

# 欠損値が存在するかを確認
print("各カラムの欠損値数:")
print(data.isna().sum())

# 欠損値を直前の値で補完（前方補完）
data = data.fillna(method="ffill")

# 前方補完で欠損値が残る場合は後方補完も行う
data = data.fillna(method="bfill")

# 補完後に再度欠損値の確認
print("欠損値の補完後の確認:")
print(data.isna().sum())



In [ ]:
import numpy as np

# 無限大の値を検出し、欠損値に置き換え
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# 欠損値と同様に補完処理を行う
data = data.fillna(method="ffill").fillna(method="bfill")


In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

# データセットの設定
max_encoder_length = 30  # 過去30日間のデータをエンコード
max_prediction_length = 7  # 7日間の予測

# データセットの構築
training = TimeSeriesDataSet(
    data,
    time_idx="time_idx",
    target="target",
    group_ids=["Stock_ID"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Stock_ID"],
    time_varying_known_reals=["time_idx", "Open", "High", "Low", "Adj Close", "Volume"],
    time_varying_unknown_reals=["target"],
)


In [ ]:
from pytorch_forecasting.data import TorchNormalizer
from pytorch_forecasting.data import GroupNormalizer
from torch.utils.data import DataLoader

# バッチサイズを設定
batch_size = 64  # 計算資源によって調整
train_dataloader = DataLoader(training, batch_size=batch_size, shuffle=True)


In [ ]:
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

# TFTモデルの初期化
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,  # 隠れ層のサイズ
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # 1週間予測する場合
    loss=QuantileLoss(),  # Quantile Lossを使用
    log_interval=10,
    reduce_on_plateau_patience=4,
)
print(f"モデルには{tft.size()}個のパラメータが含まれています")


In [ ]:
print(len(training))  # データセットのサイズを確認
print(training[0])  # 最初のサンプルを表示して、Noneが含まれていないか確認


In [ ]:
import pytorch_lightning as pl
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import QuantileLoss
from torch.utils.data import DataLoader
import torch

# データセットとデータローダーの作成
training = TimeSeriesDataSet(
    data,
    time_idx="time_idx",
    target="target",
    group_ids=["Stock_ID"],
    max_encoder_length=30,
    max_prediction_length=7,
    static_categoricals=["Stock_ID"],
    time_varying_known_reals=["time_idx", "Open", "High", "Low", "Adj Close", "Volume"],
    time_varying_unknown_reals=["target"],
)

def custom_collate_fn(batch):
    # Noneの値をフィルタリング
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        raise ValueError("Batch is empty after filtering None values.")
    return torch.utils.data.dataloader.default_collate(batch)


train_dataloader = DataLoader(training, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)

print(len(train_dataloader))  # データローダーのサイズを確認
for batch in train_dataloader:
    print(batch)  # バッチの中身を確認
    break
# TemporalFusionTransformerインスタンスの作成
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,
    loss=QuantileLoss(),
)

# TemporalFusionTransformerをLightningModuleとしてラップする
class TFTLightningModel(pl.LightningModule):
    def __init__(self, tft_model):
        super(TFTLightningModel, self).__init__()
        self.model = tft_model

    def training_step(self, batch, batch_idx):
        # モデルの予測
        y_hat = self.model(batch)
        # 損失計算
        loss = self.model.loss(y_hat, batch["target"])
        return loss

    def configure_optimizers(self):
        return self.model.configure_optimizers()

# ラップしたモデルのインスタンス作成
wrapped_tft = TFTLightningModel(tft)

# Trainerのインスタンスを作成し、トレーニングを実行
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    max_epochs=30,
    gradient_clip_val=0.1,
)

# Trainerでモデルを訓練する
trainer.fit(model=wrapped_tft, train_dataloaders=train_dataloader)

# 学習済みモデルの保存
tft.save_model("tft_stock_model.pt")
